In [1]:
#| default_exp mplots

In [2]:
#| export
import time

class Time:
    def __init__(self):
        self.time_start = None
        self.time_end = None
        self.time_total = 0

    def start(self): 
        self.time_start = time.time()
        return self.time_start

    def end(self):
        self.time_end = time.time()
        return self.time_end
        
    def duration(self):
        self.time_total=self.time_end - self.time_start
        return self.duration

    def __str__(self):
        """
        Representación en cadena de la clase Time.

        :return: Cadena que representa la instancia de Time.
        """
        if (self.time_total == 0): self.duration();
        return f"Time Start: {self.time_start}, Time End: {self.time_end}, Duration: {self.time_total}"


In [3]:
#| export
import dvats.load as load
import os

In [4]:
#| export
class GD_Mat:
    def __init__(self, id,  name, data_path = '~/data'):
        self.id = id
        self.data_path = os.path.expanduser(data_path)
        self.zip_path = os.path.join(self.data_path, name + '.zip')
        self.mat_path = os.path.join(self.data_path, name + '.mat')
        self.mats_files = None
        self.mats = None
        self.mats_df =  None
        self.num_mats = 0
        self.num_mats_extracted = 0
        
    def download_file_from_google_drive(self):
        return load.download_file_from_google_drive(self.id, self.zip_path)
        
    def get_mat_files(self):
        self.mats_files = [f for f in load.zip_contents(self.zip_path) if not f.startswith('__MACOSX/')]
        self.num_mats = len(self.mats_files)
        self.mats = self.mats_df = [None]*self.num_mats
    
    def unzip_mat(self, all_one, case = '', print_flag = False): 
        str = load.unzip_mat(all_one, self.zip_path, self.data_path, case, print_flag)
        self.get_mat_files()
        return str
        
    def mat2csv(self, case_id, print_flag = False):
        if print_flag: print("--> mat2csv", case_id)
        case = self.mats_files[case_id]
        case_path = os.path.join(self.data_path, case)
        print("Mat2csv case", case_path)
        self.mats_df[case_id] = load.mat2csv(case_path, self.data_path, print_flag)
        if print_flag: print("mat2csv", case_id, "-->")
        
    def __str__(self): 
        str = f"FileID: {self.id}\nData path: {self.data_path}\n"
        str += f"zip path: {self.zip_path}\nmat_files: {self.mats_files}"
        str +=f"\nnum_mats: {self.num_mats}"
        return str
    

In [7]:
#| hide 
file_id = '1qq1z2mVRd7PzDqX0TDAwY7BcWVjnXUfQ'
name = 'InsectData-fig11'

gd_mat = GD_Mat(file_id, name)
print(gd_mat)
gd_mat.unzip_mat('all')
print(gd_mat)
gd_mat.mat2csv(0)
print(gd_mat)
display(gd_mat.mats_df[0].head(5))

FileID: 1qq1z2mVRd7PzDqX0TDAwY7BcWVjnXUfQ
Data path: /home/macu/data
zip path: /home/macu/data/InsectData-fig11.zip
mat_files: None
num_mats: 0
FileID: 1qq1z2mVRd7PzDqX0TDAwY7BcWVjnXUfQ
Data path: /home/macu/data
zip path: /home/macu/data/InsectData-fig11.zip
mat_files: ['Insect_one_million.mat']
num_mats: 1
Mat2csv case /home/macu/data/Insect_one_million.mat
FileID: 1qq1z2mVRd7PzDqX0TDAwY7BcWVjnXUfQ
Data path: /home/macu/data
zip path: /home/macu/data/InsectData-fig11.zip
mat_files: ['Insect_one_million.mat']
num_mats: 1


,Insect_one_million
0,0.236820
1,0.236820
2,0.238040
3,0.206300
4,0.026855


In [9]:
#| export
eammon_drive_mplots = {
    'insects0': {
        id: '1qq1z2mVRd7PzDqX0TDAwY7BcWVjnXUfQ',
        name: 'InsectData-fig11'
    }
}

In [11]:
#| hide 
eammon_drive_mplots['insects0'][name]

'InsectData-fig11'